In [1]:
from dotenv import load_dotenv
load_dotenv("../webhook/.env")

True

In [6]:
# from pathlib import Path

# from mistralai import Mistral
# from mistralai import TextChunk

# from webhook.models import Invoice
# from IPython.display import Markdown, display


# class MistralAIClient:
#     def __init__(self, api_token: str):
#         self.client = Mistral(api_key=api_token)

#     def get_response(self, file_path: str):
#         """
#         Sends an image or PDF to the Mistral AI OCR API and returns structured data.
#         """
#         try:
#             file_ext = Path(file_path).suffix.lower()
#             if file_ext == '.pdf':
#                 return self.structured_pdf_ocr(file_path)
#             else:
#                 return self.structured_ocr(file_path)
#         except Exception as e:
#             raise ValueError(f"Failed to get response from Mistral AI API: {e}")

#     def structured_pdf_ocr(self, pdf_path: str) -> Invoice:
#         """
#         Process a PDF document using OCR and extract structured data.

#         Args:
#             pdf_path: Path to the PDF file to process

#         Returns:
#             Invoice object containing the extracted data

#         Raises:
#             AssertionError: If the PDF file does not exist
#         """
#         # Validate input file
#         pdf_file = Path(pdf_path)
#         assert pdf_file.is_file(), "The provided PDF path does not exist."

#         # Upload the PDF file to Mistral
#         uploaded_pdf = self.client.files.upload(
#             file={
#                 "file_name": pdf_file.name,
#                 "content": open(pdf_file, "rb"),
#             },
#             purpose="ocr"
#         )

#         # Get a signed URL for the uploaded file
#         signed_url = self.client.files.get_signed_url(file_id=uploaded_pdf.id)

#         # Process the PDF using OCR
#         ocr_response = self.client.ocr.process(
#             model="mistral-ocr-latest",
#             document={"type": "document_url", "document_url": signed_url.url}
#         )

#         # Extract text from all pages
#         all_markdown = "\n\n".join([page.markdown for page in ocr_response.pages])
#         display(Markdown(all_markdown))
#         # Parse the OCR result into a structured JSON response
#         chat_response = self.client.chat.parse(
#             model="pixtral-12b-latest",
#             messages=[
#                 {
#                     "role": "user",
#                     "content": [
#                         TextChunk(text=(
#                             f"This is the PDF's OCR in markdown:\n{all_markdown}\n.\n"
#                             "Convert this into a structured JSON response "
#                             "with the OCR contents in a sensible dictionnary."
#                         ))
#                     ]
#                 }
#             ],
#             response_format=Invoice,
#             temperature=0
#         )

#         return chat_response.choices[0].message.parsed

import os

os.environ["PYTHONPATH"] = "/home/maarten/projects/personal/colli_parser/webhook"

In [9]:
import sys
sys.path.append("/home/maarten/projects/personal/colli_parser/webhook")
from api_client import MistralAIClient

In [11]:
from api_client import MistralAIClient

In [3]:
import os
api_client = MistralAIClient(api_token=os.getenv("MISTRAL_API_TOKEN"))
result = api_client.structured_pdf_ocr("../data/Kasticket_03042025_19h42_273429311.pdf")
import polars as pl

pl.DataFrame([result.model_dump_json()])

Colruyt Food Retail N.V.
Edingensesteenweg 196 - 1500 Halle
Tel. 023452345 - www.colruyt.be
RPR Brussel
BTW-BE0716.663.615 - IBAN BE72 293025448916 SWIFT GEBABEBB
![img-0.jpeg](img-0.jpeg)

# colruyt 

![img-1.jpeg](img-1.jpeg)

Algemene verkoopsvoorwaarden:

1. Partijen aanvaarden dat elke betwisting onder de uitsluitende bevoegdheid van de rechtbanken van Brussel valt (in voorkomend geval van een der vredegerechten, zetelend in het Justitiepaleis te Brussel, naar keuze van de eiser).
2. Alle verkopen zijn strikt contant betaalbaar. In geval van niet-betaling of van een slechts
gedeeltelijke betaling, moet het niet-voldane bedrag betaald worden op de zetel van de Colruyt Food Retail N.V. te Halle, Edingensesteenweg 196, waar geldig voor verkoper kan ontvangen worden.
In geval een der partijen een of meerdere van haar voornaamste verbintenissen niet uitvoert of
slechts gedeeltelijk uitvoert, is deze vanaf de datum van haar wanprestatie van rechtswege en
zonder aanmaning een intrest van $10 \%$ per jaar verschuldigd op het niet-betaalde bedrag.
Bovendien is zij van rechtswege en zonder aanmaning een forfaitaire schadeloosstelling verschuldigd van
$10 \%$ op het betrokken bedrag, met een minimum van 25 euro per factuur, indien een maand na het begin van de
wanprestatie de betaling nog niet (volledig) werd uitgevoerd.
3. De klant die verpakkingen, voorzien van een 'groen punt', uitvoert naar een land waar
daarop rechten bestaan, dient zich in verband daarmee in regel te stellen.
4. Colruyt Food Retail N.V. heeft vrijstelling van certificatie volgens machtiging nr. 847.
5. De contractuele garantiebepalingen doen geen afbreuk aan de wettelijke garantie. U kan de bijzondere voorwaarden opvragen bij de verantwoordelijke van uw Colruytwinkel.

column_0
str
"""{""date"":""2023-10-01"",""page"":1,…"


In [5]:
df

date,page,total_amount_invoice,items
str,i64,f64,list[struct[5]]
"""2023-10-01""",1,32.5,"[{10.0,1.5,2.0,5.0,""Item 1""}, {15.0,0.75,1.0,3.0,""Item 2""}]"


In [4]:
df = pl.DataFrame([result.model_dump_json()])
df = df.select(
            pl.col("column_0").str.json_decode().alias("page_struct")
        ).unnest("page_struct")

def group_waarborg_fields(invoice_items_df: pl.DataFrame) -> pl.DataFrame:
    waarborg_filter = pl.col("description").str.contains("waarborg")
    waarborg_df = invoice_items_df.filter(waarborg_filter)
    return pl.concat(
        [
            invoice_items_df.filter(~waarborg_filter),
            waarborg_df.group_by(pl.lit(1)).agg(
                pl.exclude(["adjusted_amount"]).first(),
                pl.sum("adjusted_amount").alias("adjusted_amount")
            ).select(invoice_items_df.columns).with_columns(pl.lit("waarborg net").alias("description")),
        ]
    )

def clean_invoice_df(invoice_items_df: pl.DataFrame) -> pl.DataFrame:
    total_amount_filter = pl.col("description").str.contains(
        "total payment|total amount"
    )

    adjusted_discount = (
        pl.when(
        pl.col("next_description").str.to_lowercase().str.starts_with("korting")
        ).then(pl.col("next_discount")
        ).otherwise(
            # pl.when(pl.col("description").str.contains("korting"))
            # .then(pl.col("discount"))
            # .otherwise(pl.lit(0.0))
            pl.col("discount")
        ).alias("discount")
    )

    # First extract the total amount from any row with korting/total payment/total amount due
    invoice_items_df = (
        invoice_items_df.explode("items")
        .unnest("items")
        .filter(pl.col("description").is_not_null())
        .with_columns((pl.col("quantity")*pl.col("unit_price")).round(2).alias("price"))
        .with_columns(pl.col("description").str.to_lowercase().alias("description"))
        .with_columns([
            pl.col("discount").shift(-1).alias("next_discount"),
            pl.col("description").shift(-1).alias("next_description"),
        ])
        .with_columns(
            pl.when(pl.col("next_description").str.to_lowercase().str.starts_with("korting")).then((pl.col("description") + " "+ pl.col("next_description"))).otherwise(pl.col("description")).alias("description")
        ).with_columns(adjusted_discount)  
    )

    # Get total amount if available (use first match if multiple rows)
    total_amount_df = invoice_items_df.filter(total_amount_filter)
    total_amount = (
        total_amount_df["total_amount_invoice"][0] if not total_amount_df.is_empty() else None
    )

    # apple due to xtra sign similar to an apple
    not_a_product_filter = pl.col("description").str.contains(
        "total payment|total amount|apple|maestro"
    )
    cleaned_df = (
        invoice_items_df.filter(~not_a_product_filter)
        # Adjust price by discount
        .with_columns((pl.col("price")*(1 - (pl.col("discount")/100))).round(2).alias("adjusted_amount"))
    )

    # Add total_amount as a column and check for discrepancy
    sum_price = cleaned_df["adjusted_amount"].sum()
    if total_amount is not None and abs(sum_price - total_amount) > 0.01:
        print(
            f"Sum of items ({sum_price}) differs from total amount ({total_amount})"
        )

    return group_waarborg_fields(
        cleaned_df.with_columns(pl.lit(total_amount).alias("total_amount"))
    )
clean_invoice_df(df).select("price", "discount","quantity", "description", "adjusted_amount").to_pandas()

,price,discount,quantity,description,adjusted_amount
0,20.0,5.0,2.0,item 1,19.00
1,15.0,3.0,1.0,item 2,14.55
